<a href="https://colab.research.google.com/github/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/blob/main/Code/Updated__New_Notebook_Blancpain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DMML TEAM BLANCPAIN

##Import some librairies

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy
from gensim.models import Word2Vec
pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

%matplotlib inline
sns.set_style("dark")

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

##Data Importation##

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/training_data.csv')

df_test = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/test_data.csv')

df_sample = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/sample_submission.csv')

##Data visualization##

###Training set

In [ ]:
df_train

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1
...,...,...,...,...,...
6466,4377,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1
6467,3408,derail,NaN,@EmiiliexIrwin Totally agree.She is 23 and kno...,0
6468,9794,trapped,NaN,Hollywood Movie About Trapped Miners Released ...,1
6469,10344,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1


In [ ]:
print("There are {0} rows and {1} columns in the train dataset.".format(df_train.shape[0],df_train.shape[1]))

There are 6471 rows and 5 columns in the train dataset.


###Test Set

In [ ]:
df_test

,id,keyword,location,text
0,9972,tsunami,NaN,Crptotech tsunami and banks.\n http://t.co/KHz...
1,9865,traumatised,"Portsmouth, UK",I'm that traumatised that I can't even spell p...
2,1937,burning%20buildings,NaN,@foxnewsvideo @AIIAmericanGirI @ANHQDC So ... ...
3,3560,desolate,NaN,Me watching Law &amp; Order (IB: @sauldale305)...
4,2731,crushed,bahstun/porta reeko,Papi absolutely crushed that ball
...,...,...,...,...
1137,3497,derailed,NaN,@ItsQueenBaby I'm at work it's a bunch of ppl ...
1138,9191,suicide%20bomber,NaN,#?? #?? #??? #??? Suicide bomber kills 15 in S...
1139,10217,volcano,NaN,Eruption of Indonesian volcano sparks transpor...
1140,5308,fear,New York,Never let fear get in the way of achieving you...


In [ ]:
print("There are {0} rows and {1} columns in the test dataset.".format(df_test.shape[0],df_test.shape[1]))

There are 1142 rows and 4 columns in the test dataset.


In [ ]:
df_sample.head()

,target
0,0
1,0
2,0
3,0
4,0


###Drop Null values

In [ ]:
df_train.fillna("Unknown", inplace = True)
df_test.fillna("Unknown", inplace = True)

###Base Rate 

In [ ]:
base_rate = max(len(df_train[df_train["target"] == 0]) / len(df_train), len(df_train[df_train["target"] == 1]) / len(df_train))
print("The base rate for this problem is :", base_rate)

The base rate for this problem is : 0.5719363313243703


##Processing

In [ ]:
#import emojis library
!pip install emot
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

#removing emojis function (emoticons displayed in text)
def convert_emoticons_text(dataFrame):
  len = dataFrame.shape[0]
  i=0
  while i<len:
    for emot in EMOTICONS:
        dataFrame.iloc[i] = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), dataFrame.iloc[i])
    i+=1
  return dataFrame.head(20)

#removing emojis function (emoticons displayed in emojis)
def convert_emoticons_emoji(dataFrame):
  len = dataFrame.shape[0]
  i=0
  while i<len:
    for emot in UNICODE_EMO:
        dataFrame.iloc[i] = dataFrame.iloc[i].replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    i+=1
  return dataFrame.head(20)

In [ ]:
#clean smileys in text
#convert_emoticons(df_train['text'])

In [ ]:
df_train["keyword_text"] = df_train['keyword'] + ' ' + df_train['text'] 
df_train["keyword_text"] = df_train["keyword_text"].astype(str)
df_test["keyword_text"] = df_test['keyword'] + ' ' + df_test['text'] 
df_test["keyword_text"] = df_test["keyword_text"].astype(str)

In [ ]:
keyword_list = ['panicking',
 'suicide%20bombing',
 'body%20bags',
 'blight',
 'armageddon',
 'stretcher',
 'razed',
 'ruin',
 'wreckage',
 'derailment',
 'bombing',
 'nuclear%20disaster',
 'panic',
 'obliteration',
 'blazing',
 'typhoon',
 'outbreak',
 'wrecked',
 'oil%20spill',
 'rescuers',
 'debris',
 'wildfire',
 'aftershock',
 'meltdown',
 'bloody',
 'traumatised',
 'electrocute',
 'smoke',
 'screaming',
 'body%20bag',
 'suicide%20bomber',
 'blew%20up']

In [ ]:
def clean_keword_text(row):
  if row['keyword'] in keyword_list:
    return row['keyword'] + ' ' + row['text']
  else:
    return row['text']

df_train["keyword_cleaned_text"] = df_train.apply(clean_keword_text, axis=1)
df_test["keyword_cleaned_text"] = df_test.apply(clean_keword_text, axis=1)

In [ ]:
df_train.head()

,id,keyword,location,text,target,keyword_text,keyword_cleaned_text
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0,destroyed Black Eye 9: A space battle occurred...,Black Eye 9: A space battle occurred at Star O...
1,853,bioterror,Unknown,#world FedEx no longer to transport bioterror ...,0,bioterror #world FedEx no longer to transport ...,#world FedEx no longer to transport bioterror ...
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1,windstorm Reality Training: Train falls off el...,Reality Training: Train falls off elevated tra...
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1,hazardous #Taiwan Grace: expect that large roc...,#Taiwan Grace: expect that large rocks trees m...
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1,hostage New ISIS Video: ISIS Threatens to Behe...,New ISIS Video: ISIS Threatens to Behead Croat...


# Processing function

In [ ]:
# Load English language model
sp = spacy.load('en_core_web_sm')

# Create a list of punctuation marks
punctuations = string.punctuation

# Create a list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Create a list of isolated letters
lonely_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Create a list of further elements that could be deleted to improve the model
to_be_deleted = ['http','html','@']

def tokenize_function(text):
    # Create token object, which is used to create documents with linguistic annotations.
    sp_obj = sp(text)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in sp_obj ]

    # Correcting spelling mistakes
    #mytokens = [ spell(str(word)) for word in sp_obj]

    ## Remove stop words
    #mytokens = [ word for word in mytokens if word not in stop_words]

    # Remove punctuation
    #mytokens = [ word for word in mytokens if word not in punctuations]

    # Remove isolated letters
    #mytokens = [ word for word in mytokens if word not in lonely_letters]

    # Delete further elements
    #mytokens = [ word for word in mytokens if all(char not in word for char in to_be_deleted)]
    
    return mytokens

In [ ]:
tokenize_function("j'aime le machine learning788")

["j'aime", 'le', 'machine', 'learning788']

# Create ngrams on text feature

In [ ]:
import gensim.downloader
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

def eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', model_type='randomForest', feature_engineering='tfidf'):
  ''' Eval pipeline '''

  # Make copies to avoid any changes in the main datasets
  train = df_train.copy()
  test = df_test.copy()

  # Define the model
  if model_type == 'logisticRegression':
    model = LogisticRegression(solver='lbfgs', #LogisticRegressionCV
                                #cv=2, 
                                max_iter=1000, 
                                random_state=50,
                                n_jobs=multiprocessing.cpu_count())
  
  elif model_type == 'randomForest':
    model = RandomForestClassifier(n_estimators=200,
                                   max_depth=8,
                                   random_state=50,
                                   n_jobs=multiprocessing.cpu_count())

  elif model_type == 'knn':
    model = KNeighborsClassifier(random_state=50,
                                 n_jobs=multiprocessing.cpu_count())

  # Define embeddings and model
  if feature_engineering == 'tfidf':
    feature_vector = TfidfVectorizer(tokenizer=tokenize_function)
    model = Pipeline([('vectorizer', feature_vector), 
                   ('classifier', model)])
  
  elif feature_engineering == 'w2v':

    # Function that uses w2v embeddings and compute the mean value of all the of w2v vectors of all the words
    def document_vector(doc):
      """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
      tokens = tokenize_function(doc)
      doc = [word for word in tokens if word in feature_vector.wv.vocab]
      return np.mean(feature_vector[doc], axis=0)

    # Create corpus for w2v
    sentences = []
    for text in [list(train[target_col]), list(test[target_col])]:
      for string in text:
        sentences.append(tokenize_function(string))

    # Create w2v embeddings
    feature_vector = gensim.models.Word2Vec(sentences, 
                                            size=100, 
                                            window=10, 
                                            min_count=1, 
                                            iter=20,
                                            seed=50,
                                            workers=multiprocessing.cpu_count()
                                            )

    train[target_col] = train[target_col].apply(document_vector)
    test[target_col] = test[target_col].apply(document_vector)
  
  # Split dataset
  df_train_eval, df_validation = train_test_split(train, test_size=0.2, random_state = 50)

  y_train_eval = df_train_eval[["target"]]
  y_validation = df_validation[["target"]]

  X1_train_eval = list(df_train_eval[target_col])
  X1_validation = list(df_validation[target_col])

  # Fit data
  model.fit(X1_train_eval, y_train_eval)

  # Eval data
  train_accuracy_score = accuracy_score(y_train_eval, model.predict(X1_train_eval))
  print(f"The training accuracy is : {train_accuracy_score}")

  validation_accuracy_score = accuracy_score(y_validation, model.predict(X1_validation))
  print(f"The validation accuracy is : {validation_accuracy_score}")

  # Retrain on the full dataset
  y_train = train[["target"]]
  X1_train = list(train[target_col])
  X1_test = list(test[target_col])

  # Create submission file
  model.fit(X1_train, y_train)
  submission = pd.DataFrame(model.predict(X1_test), columns=["target"])
  submission.to_csv(f'submission_{target_col}_{feature_engineering}_model_{model_type}_test2.csv', index=False)

###With "Keyword" feature

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', model_type='randomForest', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', model_type='randomForest', feature_engineering='w2v')

The training accuracy is : 0.7436244204018547
The validation accuracy is : 0.6988416988416989
The training accuracy is : 0.7436244204018547
The validation accuracy is : 0.6988416988416989


In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', model_type='logisticRegression', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', model_type='logisticRegression', feature_engineering='w2v')

###With "Text" Feature

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', model_type='randomForest', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', model_type='randomForest', feature_engineering='w2v')

The training accuracy is : 0.6477975270479135
The validation accuracy is : 0.6471042471042471
The training accuracy is : 0.8427357032457496
The validation accuracy is : 0.7637065637065638


In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', model_type='logisticRegression', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', model_type='logisticRegression', feature_engineering='w2v')

###With "Text" and "Keyword" features

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', model_type='randomForest', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', model_type='randomForest', feature_engineering='w2v')

The training accuracy is : 0.6549459041731066
The validation accuracy is : 0.6548262548262548
The training accuracy is : 0.857225656877898
The validation accuracy is : 0.7760617760617761


In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', model_type='logisticRegression', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', model_type='logisticRegression', feature_engineering='w2v')

# With "Text" and "Keyword" cleaned

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_cleaned_text', model_type='randomForest', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_cleaned_text', model_type='randomForest', feature_engineering='w2v')

The training accuracy is : 0.6505023183925811
The validation accuracy is : 0.6501930501930502
The training accuracy is : 0.8543276661514683
The validation accuracy is : 0.7791505791505792


In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_cleaned_text', model_type='logisticRegression', feature_engineering='tfidf')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_cleaned_text', model_type='logisticRegression', feature_engineering='w2v')